In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

## Upload census geometry data

In [3]:
ur_files = ddf.read_csv('county_shp.csv', dtype = {'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'NAME': str, 'NAMELSAD': str})
census = ur_files.compute()
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33...."
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999..."
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999..."
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71..."
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4..."


In [4]:
census.shape

(85230, 14)

In [5]:
from shapely import wkt

In [6]:
def findCentroid(g):
    p1 = wkt.loads(g)
    return p1.centroid.wkt

In [7]:
census['Centroid'] = census['geometry'].apply(findCentroid)

In [8]:
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Centroid
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33....",POINT (-82.42777455266631 33.02009497835021)
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999...",POINT (-80.9680558426884 35.45990205837115)
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999...",POINT (-71.03573713399868 42.37754679162973)
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71...",POINT (-87.71008151357118 41.91187356531011)
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4...",POINT (-122.9834298654585 44.96158272247879)


## upload temp count data - 2009

In [9]:
ur_files = ddf.read_csv('Temp_Count_New/Temp_Count_2020.csv', dtype={'station': 'object'})
temp = ur_files.compute()
temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
temp.head()

,station,latitude,longitude,mean_temp_below_30,mean_temp_30-40,mean_temp_40-50,mean_temp_50-60,mean_temp_60-70,mean_temp_70-80,mean_temp_80-90,...,min_temp_over_90,max_temp_below_30,max_temp_30-40,max_temp_40-50,max_temp_50-60,max_temp_60-70,max_temp_70-80,max_temp_80-90,max_temp_over_90,year
0,10000199999,51.602414,6.142172,0,25,86,58,64,21,8,...,0,0,9,52,78,54,41,18,9,2020
1,1001099999,70.933333,-8.666667,113,185,67,0,0,0,0,...,0,67,177,116,5,0,0,0,0,2020
2,1001499999,59.791925,5.340850,0,41,116,83,33,8,0,...,0,0,18,109,79,53,11,5,0,2020
3,10018099999,54.913250,8.340472,0,25,118,94,70,16,0,...,0,0,8,101,78,91,21,13,0,2020
4,1002099999,80.050000,16.250000,145,80,21,3,0,0,0,...,0,133,72,37,6,1,0,0,0,2020


In [10]:
def findLon(point):
    strList = point.split(' ')
    lon = strList[1][1:]
    lon = float(lon)
    return lon

In [11]:
def findLat(point):
    strList = point.split(' ')
    lat = strList[2][:-1]
    lat = float(lat)
    return lat

In [12]:
census['lat'] = census['Centroid'].apply(findLat)
census['lon'] = census['Centroid'].apply(findLon)

In [13]:
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Centroid,lat,lon
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33....",POINT (-82.42777455266631 33.02009497835021),33.020095,-82.427775
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999...",POINT (-80.9680558426884 35.45990205837115),35.459902,-80.968056
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999...",POINT (-71.03573713399868 42.37754679162973),42.377547,-71.035737
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71...",POINT (-87.71008151357118 41.91187356531011),41.911874,-87.710082
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4...",POINT (-122.9834298654585 44.96158272247879),44.961583,-122.983430


In [14]:
def ID_Coordinate(ID,lat, lon):
    setList = []
    setList.append(ID)
    setList.append(lat)
    setList.append(lon)
    return setList

In [15]:
temp['ID_Coordinate'] = temp.apply(lambda x: ID_Coordinate(x['station'], x['latitude'], x['longitude']), axis=1)

In [16]:
def dist(p0, p1):
    return (((p0[0] - p1[0])**2) + ((p0[1] - p1[1])**2))**.5

In [17]:
pointList = temp['ID_Coordinate'].tolist()

In [18]:
len(pointList)

12259

In [19]:
pointList[:5]

[['10000199999', 51.602414, 6.142172],
 ['1001099999', 70.9333333, -8.6666667],
 ['1001499999', 59.791925, 5.34085],
 ['10018099999', 54.91325, 8.340472],
 ['1002099999', 80.05, 16.25]]

In [20]:
def findSquare(pointList, lat, lon):
    square = []
    for point in pointList:
        point_lat = point[1]
        point_lon = point[2]
        if point_lat < lat + 0.5 and point_lat > lat - 0.5 and point_lon < lon + 0.5 and point_lon > lon - 0.5:
            square.append(point)
        
    return square

In [21]:
result1 = findSquare(pointList,33.020095, -82.427775)
result1

[['72095100330', 32.609, -82.37],
 ['72218003820', 33.3644, -81.9633],
 ['72218113837', 33.46694, -82.03861]]

In [22]:
def shortestDistance(lat, lon):
    p1 = []
    p1.append(lat)
    p1.append(lon)
    distants = []
    square = findSquare(pointList, lat, lon)
    for point in square:
        p_id = point[0]
        p_lat = point[1]
        p_lon = point[2]
        p2 = []
        p2.append(p_lat)
        p2.append(p_lon)
        distant = dist(p1, p2)
        dist_id = []
        dist_id.append(p_id)
        dist_id.append(distant)
        distants.append(dist_id)
    distantList = []    
    for ele in distants:
        distantList.append(ele[1])
    for item in distants:
        if item[1] == min(distantList):
            return item[0]

In [23]:
census['NearestStation'] = census.apply(lambda x: shortestDistance(x['lat'], x['lon']), axis =1)

In [24]:
census = census.rename(columns = {'NearestStation': 'station'})

In [25]:
census_temp = census.merge(temp, on = 'station', how = 'left')
census_temp.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,...,max_temp_below_30,max_temp_30-40,max_temp_40-50,max_temp_50-60,max_temp_60-70,max_temp_70-80,max_temp_80-90,max_temp_over_90,year,ID_Coordinate
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,...,0.0,0.0,7.0,29.0,62.0,85.0,116.0,67.0,2020.0,"[72095100330, 32.609, -82.37]"
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,...,0.0,3.0,23.0,61.0,81.0,74.0,95.0,29.0,2020.0,"[72212853899, 35.48306, -81.16111]"
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,...,4.0,34.0,73.0,66.0,61.0,61.0,50.0,13.0,2020.0,"[99497199999, 42.35, -71.05]"
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,...,0.0,1.0,14.0,16.0,12.0,20.0,1.0,0.0,2020.0,"[99849799999, 41.976, -87.648]"
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,...,0.0,2.0,31.0,103.0,83.0,70.0,53.0,24.0,2020.0,"[72694024232, 44.905, -123.0011]"


In [26]:
census_temp.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'GEOID', 'NAME',
       'NAMELSAD', 'STUSPS', 'NAMELSADCO', 'STATE_NAME', 'LSAD', 'ALAND',
       'AWATER', 'geometry', 'Centroid', 'lat', 'lon', 'station', 'latitude',
       'longitude', 'mean_temp_below_30', 'mean_temp_30-40', 'mean_temp_40-50',
       'mean_temp_50-60', 'mean_temp_60-70', 'mean_temp_70-80',
       'mean_temp_80-90', 'mean_temp_over_90', 'min_temp_below_30',
       'min_temp_30-40', 'min_temp_40-50', 'min_temp_50-60', 'min_temp_60-70',
       'min_temp_70-80', 'min_temp_80-90', 'min_temp_over_90',
       'max_temp_below_30', 'max_temp_30-40', 'max_temp_40-50',
       'max_temp_50-60', 'max_temp_60-70', 'max_temp_70-80', 'max_temp_80-90',
       'max_temp_over_90', 'year', 'ID_Coordinate'],
      dtype='object')

In [27]:
temp_census = census_temp[['STATEFP', 'COUNTYFP', 'TRACTCE', 'station', 'year']]
temp_census.head()

,STATEFP,COUNTYFP,TRACTCE,station,year
0,13,163,960300,72095100330,2020.0
1,37,109,071201,72212853899,2020.0
2,25,025,050200,99497199999,2020.0
3,17,031,230200,99849799999,2020.0
4,41,047,001607,72694024232,2020.0


In [28]:
temp_census.to_csv('temp_station/temp_station_2020.csv')

In [29]:
census_temp['fips'] = census_temp['STATEFP'] + census_temp['COUNTYFP']
temp_count = census_temp[['fips','mean_temp_below_30', 'mean_temp_30-40', 'mean_temp_40-50', 
                          'mean_temp_50-60', 'mean_temp_60-70', 'mean_temp_70-80', 
                          'mean_temp_80-90', 'mean_temp_over_90', 'min_temp_below_30', 
                          'min_temp_30-40', 'min_temp_40-50', 'min_temp_50-60', 
                          'min_temp_60-70', 'min_temp_70-80', 'min_temp_80-90', 
                          'min_temp_over_90','max_temp_below_30', 'max_temp_30-40', 
                          'max_temp_40-50', 'max_temp_50-60', 'max_temp_60-70', 
                          'max_temp_70-80','max_temp_80-90', 'max_temp_over_90','year']]
temp_count.head()

,fips,mean_temp_below_30,mean_temp_30-40,mean_temp_40-50,mean_temp_50-60,mean_temp_60-70,mean_temp_70-80,mean_temp_80-90,mean_temp_over_90,min_temp_below_30,...,min_temp_over_90,max_temp_below_30,max_temp_30-40,max_temp_40-50,max_temp_50-60,max_temp_60-70,max_temp_70-80,max_temp_80-90,max_temp_over_90,year
0,13163,0.0,7.0,43.0,51.0,91.0,118.0,56.0,0.0,4.0,...,0.0,0.0,0.0,7.0,29.0,62.0,85.0,116.0,67.0,2020.0
1,37109,3.0,36.0,63.0,74.0,82.0,102.0,6.0,0.0,45.0,...,0.0,0.0,3.0,23.0,61.0,81.0,74.0,95.0,29.0,2020.0
2,25025,15.0,60.0,87.0,59.0,60.0,66.0,15.0,0.0,36.0,...,0.0,4.0,34.0,73.0,66.0,61.0,61.0,50.0,13.0,2020.0
3,17031,0.0,8.0,21.0,17.0,16.0,2.0,0.0,0.0,1.0,...,0.0,0.0,1.0,14.0,16.0,12.0,20.0,1.0,0.0,2020.0
4,41047,0.0,40.0,112.0,97.0,84.0,32.0,1.0,0.0,39.0,...,0.0,0.0,2.0,31.0,103.0,83.0,70.0,53.0,24.0,2020.0


In [30]:
example = temp_count[temp_count['fips'] == '13163']
example

,fips,mean_temp_below_30,mean_temp_30-40,mean_temp_40-50,mean_temp_50-60,mean_temp_60-70,mean_temp_70-80,mean_temp_80-90,mean_temp_over_90,min_temp_below_30,...,min_temp_over_90,max_temp_below_30,max_temp_30-40,max_temp_40-50,max_temp_50-60,max_temp_60-70,max_temp_70-80,max_temp_80-90,max_temp_over_90,year
0,13163,0.0,7.0,43.0,51.0,91.0,118.0,56.0,0.0,4.0,...,0.0,0.0,0.0,7.0,29.0,62.0,85.0,116.0,67.0,2020.0
67036,13163,0.0,18.0,51.0,64.0,78.0,101.0,38.0,0.0,23.0,...,0.0,0.0,0.0,13.0,40.0,59.0,87.0,94.0,57.0,2020.0
77290,13163,0.0,7.0,43.0,51.0,91.0,118.0,56.0,0.0,4.0,...,0.0,0.0,0.0,7.0,29.0,62.0,85.0,116.0,67.0,2020.0
82423,13163,0.0,18.0,51.0,64.0,78.0,101.0,38.0,0.0,23.0,...,0.0,0.0,0.0,13.0,40.0,59.0,87.0,94.0,57.0,2020.0


In [31]:
temp_count = temp_count.dropna()

In [32]:
temp_count = temp_count.groupby(['fips']).mean()
temp_count = temp_count.reset_index()
temp_count.head()

,fips,mean_temp_below_30,mean_temp_30-40,mean_temp_40-50,mean_temp_50-60,mean_temp_60-70,mean_temp_70-80,mean_temp_80-90,mean_temp_over_90,min_temp_below_30,...,min_temp_over_90,max_temp_below_30,max_temp_30-40,max_temp_40-50,max_temp_50-60,max_temp_60-70,max_temp_70-80,max_temp_80-90,max_temp_over_90,year
0,01001,0.176471,10.882353,41.941176,59.823529,85.705882,122.647059,44.823529,0.0,14.294118,...,0.0,0.0,0.176471,7.411765,31.470588,58.823529,95.352941,110.647059,62.117647,2020.0
1,01003,0.000000,3.023256,18.790698,34.744186,76.372093,106.534884,64.604651,0.0,3.139535,...,0.0,0.0,0.000000,3.093023,17.023256,41.116279,90.069767,129.651163,23.116279,2020.0
2,01005,0.000000,7.111111,19.000000,31.444444,39.000000,50.222222,7.333333,0.0,9.333333,...,0.0,0.0,0.000000,1.777778,13.888889,30.000000,47.888889,49.111111,11.444444,2020.0
3,01007,1.250000,13.625000,46.250000,69.500000,84.875000,121.125000,26.875000,0.0,15.750000,...,0.0,0.0,1.375000,10.625000,39.250000,64.125000,91.750000,108.625000,47.750000,2020.0
4,01009,3.000000,20.375000,57.375000,72.625000,83.500000,106.437500,19.375000,0.0,22.625000,...,0.0,0.0,2.687500,14.500000,52.750000,76.062500,84.062500,99.375000,33.187500,2020.0


In [33]:
temp_count['mean_temp_below_30'] = round(temp_count['mean_temp_below_30'])
temp_count['mean_temp_30-40'] = round(temp_count['mean_temp_below_30'])
temp_count['mean_temp_40-50'] = round(temp_count['mean_temp_40-50'])
temp_count['mean_temp_50-60'] = round(temp_count['mean_temp_50-60'])
temp_count['mean_temp_60-70'] = round(temp_count['mean_temp_60-70'])
temp_count['mean_temp_70-80'] = round(temp_count['mean_temp_70-80'])
temp_count['mean_temp_80-90'] = round(temp_count['mean_temp_80-90'])
temp_count['mean_temp_over_90'] = round(temp_count['mean_temp_over_90'])

temp_count['min_temp_below_30'] = round(temp_count['min_temp_below_30'])
temp_count['min_temp_30-40'] = round(temp_count['min_temp_below_30'])
temp_count['min_temp_40-50'] = round(temp_count['min_temp_40-50'])
temp_count['min_temp_50-60'] = round(temp_count['min_temp_50-60'])
temp_count['min_temp_60-70'] = round(temp_count['min_temp_60-70'])
temp_count['min_temp_70-80'] = round(temp_count['min_temp_70-80'])
temp_count['min_temp_80-90'] = round(temp_count['min_temp_80-90'])
temp_count['min_temp_over_90'] = round(temp_count['min_temp_over_90'])

temp_count['max_temp_below_30'] = round(temp_count['max_temp_below_30'])
temp_count['max_temp_30-40'] = round(temp_count['max_temp_below_30'])
temp_count['max_temp_40-50'] = round(temp_count['max_temp_40-50'])
temp_count['max_temp_50-60'] = round(temp_count['max_temp_50-60'])
temp_count['max_temp_60-70'] = round(temp_count['max_temp_60-70'])
temp_count['max_temp_70-80'] = round(temp_count['max_temp_70-80'])
temp_count['max_temp_80-90'] = round(temp_count['max_temp_80-90'])
temp_count['max_temp_over_90'] = round(temp_count['max_temp_over_90'])

In [34]:
temp_count.head()

,fips,mean_temp_below_30,mean_temp_30-40,mean_temp_40-50,mean_temp_50-60,mean_temp_60-70,mean_temp_70-80,mean_temp_80-90,mean_temp_over_90,min_temp_below_30,...,min_temp_over_90,max_temp_below_30,max_temp_30-40,max_temp_40-50,max_temp_50-60,max_temp_60-70,max_temp_70-80,max_temp_80-90,max_temp_over_90,year
0,01001,0.0,0.0,42.0,60.0,86.0,123.0,45.0,0.0,14.0,...,0.0,0.0,0.0,7.0,31.0,59.0,95.0,111.0,62.0,2020.0
1,01003,0.0,0.0,19.0,35.0,76.0,107.0,65.0,0.0,3.0,...,0.0,0.0,0.0,3.0,17.0,41.0,90.0,130.0,23.0,2020.0
2,01005,0.0,0.0,19.0,31.0,39.0,50.0,7.0,0.0,9.0,...,0.0,0.0,0.0,2.0,14.0,30.0,48.0,49.0,11.0,2020.0
3,01007,1.0,1.0,46.0,70.0,85.0,121.0,27.0,0.0,16.0,...,0.0,0.0,0.0,11.0,39.0,64.0,92.0,109.0,48.0,2020.0
4,01009,3.0,3.0,57.0,73.0,84.0,106.0,19.0,0.0,23.0,...,0.0,0.0,0.0,14.0,53.0,76.0,84.0,99.0,33.0,2020.0


In [35]:
temp_count.to_csv('Temp_Count_New/temp_count_county_level_2020.csv')